<a href="https://colab.research.google.com/github/ricospeloacaso/python_para_investimentos/blob/master/17_CVM_Os_Melhores_e_os_Piores_Fundos_de_Investimento_de_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Base de Dados: http://dados.cvm.gov.br/




###Bibliotecas

In [0]:
import pandas as pd
from pandas.tseries.offsets import BDay
from datetime import date
pd.set_option("display.max_colwidth", 150)

###Funções para Coleta e Tratamento de dados de Fundos de Investimento fornecidos pela CVM

In [0]:
def busca_cadastro_cvm(data=(date.today()-BDay(2))):
  if data is not busca_cadastro_cvm.__defaults__[0]:
    data = pd.to_datetime(data)
  
  try:
    url = 'http://dados.cvm.gov.br/dados/FI/CAD/DADOS/inf_cadastral_fi_{}{:02d}{:02d}.csv'.format(data.year, data.month, data.day)
    return pd.read_csv(url, sep=';', encoding='ISO-8859-1')

  except: 
    print("Arquivo {} não encontrado!".format(url))
    print("Forneça outra data!")

In [0]:
def busca_informes_diarios_cvm_por_periodo(data_inicio, data_fim):
  datas = pd.date_range(data_inicio, data_fim, freq='MS') 
  informe_completo = pd.DataFrame()

  for data in datas:
    try:
      url ='http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{:02d}.csv'.format(data.year, data.month)
      informe_mensal = pd.read_csv(url, sep=';')    
    
    except: 
      print("Arquivo {} não encontrado!".format(url))    

    informe_completo = pd.concat([informe_completo, informe_mensal], ignore_index=True)

  return informe_completo

In [0]:
def melhores_e_piores(informes, cadastro, top=5, minimo_de_cotistas=100, classe=''):  
  cadastro      = cadastro[cadastro['SIT'] == 'EM FUNCIONAMENTO NORMAL']
  fundos        = informes[informes['NR_COTST'] >= minimo_de_cotistas]
  cnpj_informes = fundos['CNPJ_FUNDO'].drop_duplicates()
  
  fundos = fundos.pivot(index='DT_COMPTC', columns='CNPJ_FUNDO')  
  cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]
  
  if classe == 'multimercado':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Multimercado']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'acoes':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Ações']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'rendafixa':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Renda Fixa']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'cambial':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Cambial']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
  
  #melhores
  melhores = pd.DataFrame()
  melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False)[:top] - 1) * 100
  for cnpj in melhores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    melhores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    melhores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]

  #piores
  piores = pd.DataFrame()
  piores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=True)[:top] - 1) * 100
  for cnpj in piores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    piores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    piores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    piores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]
  
  return melhores, piores

###Buscando Dados da CVM

In [0]:
cadastro = busca_cadastro_cvm()

In [0]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio='2020-01', data_fim='2020-05')

In [0]:
cadastro

In [0]:
informes

###Fundos de Investimento: 10 Melhores e 10 Piores de 2020

In [0]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100)

In [0]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
34.979.818/0001-10,59.694443,BTG PACTUAL OURO USD FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,1.254679e+08
09.601.190/0001-77,53.340957,ÓRAMA OURO FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,3.736876e+08
33.925.149/0001-30,52.668225,VITREO OURO FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,2.462894e+08
24.454.718/0001-16,47.552244,DOLAR GLOBAL MACRO OPPORTUNITIES FUNDO DE INVESTIMENTO MULTIMERCADO INVESTIMENTO NO EXTERIOR,Fundo Multimercado,9.939867e+07
33.144.687/0001-98,43.460456,BTG PACTUAL DIGITAL DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.838957e+08
01.623.535/0001-81,43.153661,ITAÚ CAMBIAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,1.076893e+09
28.516.058/0001-01,43.140339,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL ÁGORA,Fundo Cambial,1.655603e+07
31.132.386/0001-19,43.114212,TREND DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.292850e+08
01.430.938/0001-04,43.096075,BTG PACTUAL DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.790911e+08


In [0]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
24.837.071/0001-01,-75.899511,WM DELTA CRÉDITO ESTRUTURADO FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO - CRÉDITO PRIVADO,Fundo Multimercado,1.107619e+06
26.648.868/0001-96,-63.445553,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES II - BDR NÍVEL I,Fundo de Ações,3.411718e+08
12.987.743/0001-86,-63.377084,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I,Fundo de Ações,1.371619e+09
31.936.857/0001-41,-62.783939,TREND IBOVESPA ALAVANCADO FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,8.549544e+07
10.869.628/0001-81,-56.136318,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Fundo de Ações,1.815607e+07
10.590.125/0001-72,-55.825129,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES CIELO,Fundo de Ações,5.942560e+06
11.182.072/0001-13,-53.975565,LOGOS TOTAL RETURN FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,1.507532e+08
32.319.324/0001-83,-53.694348,TREND IBOVESPA DOLAR FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,7.348693e+06
10.551.375/0001-01,-50.176998,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA CONSTRUÇÃO CIVIL,Fundo de Ações,3.788286e+08


###Fundos de Ações: 10 Melhores e 10 Piores de 2020

In [0]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100, classe='acoes')

In [0]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
19.831.126/0001-36,35.628645,WESTERN ASSET FUNDO DE INVESTIMENTO AÇÕES BDR NÍVEL I,Fundo de Ações,6.145908e+08
22.632.237/0001-28,29.728232,BB AÇÕES GLOBAIS FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I,Fundo de Ações,2.461624e+08
17.503.172/0001-80,29.717323,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA BDR NÍVEL I,Fundo de Ações,2.917604e+08
34.028.082/0001-02,28.894926,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES BDR NÍVEL I PLUS,Fundo de Ações,5.168115e+07
13.401.224/0001-57,28.473023,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES BDR NÍVEL I,Fundo de Ações,2.350280e+08
31.042.960/0001-48,27.784511,R8 BOLSA AMERICANA FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,6.925156e+06
19.436.818/0001-80,24.349691,SAFRA CONSUMO AMERICANO FUNDO DE INVESTIMENTO EM AÇÕES BDR-NÍVEL I PB,Fundo de Ações,9.741794e+08
19.436.835/0001-17,23.891509,SAFRA CONSUMO AMERICANO FDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES BDR-NÍVEL I,Fundo de Ações,6.677639e+08
21.347.643/0001-86,23.282277,BRADESCO FIC DE FUNDOS DE INVESTIMENTO EM AÇÕES GLOBAL INVESTIMENTO NO EXTERIOR,Fundo de Ações,5.159633e+07


In [0]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
26.648.868/0001-96,-63.445553,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES II - BDR NÍVEL I,Fundo de Ações,3.411718e+08
12.987.743/0001-86,-63.377084,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I,Fundo de Ações,1.371619e+09
31.936.857/0001-41,-62.783939,TREND IBOVESPA ALAVANCADO FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,8.549544e+07
10.869.628/0001-81,-56.136318,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Fundo de Ações,1.815607e+07
10.590.125/0001-72,-55.825129,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES CIELO,Fundo de Ações,5.942560e+06
32.319.324/0001-83,-53.694348,TREND IBOVESPA DOLAR FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,7.348693e+06
10.551.375/0001-01,-50.176998,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA CONSTRUÇÃO CIVIL,Fundo de Ações,3.788286e+08
09.648.050/0001-54,-50.048546,BB AÇÕES CONSTRUÇÃO CIVIL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo de Ações,2.693042e+08
30.068.271/0001-40,-48.961611,CAIXA FUNDO DE INVESTIMENTO EM AÇÕES BANCO DO BRASIL PLUS,Fundo de Ações,1.018523e+08


###Fundos de Renda Fixa: 10 Melhores e 10 Piores de 2020

In [0]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100, classe='rendafixa')

In [0]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
30.606.402/0001-03,40.403400,BB RENDA FIXA LONGO PRAZO BONDS GLOBAIS FX PRIVATE CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR FIC FI,Fundo de Renda Fixa,4.874773e+07
00.360.293/0001-18,38.751225,BB RENDA FIXA DIVIDA EXTERNA MIL FUNDO INVESTIMENTO,Fundo de Renda Fixa,2.064182e+08
26.751.304/0001-84,34.871185,BB RENDA FIXA LP CRÉDITO PRIVADO FX BONDS BRASILEIROS INVESTIMENTO NO EXTERIOR PRIVATE FIC FI,Fundo de Renda Fixa,4.380533e+07
04.240.128/0001-83,10.592442,VOTORANTIM FUNDO DE INVESTIMENTO VINTAGE RENDA FIXA - CRÉDITO PRIVADO,Fundo de Renda Fixa,5.861819e+07
06.114.630/0001-73,9.191805,SANTOS IV LQ FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,1.668154e+06
04.570.232/0001-36,8.390369,MB MATIC RENDA FIXA LONGO PRAZO FUNDO DE INVESTIMENTO FINANCEIRO,Fundo de Renda Fixa,2.168682e+06
24.022.566/0001-82,5.216968,BRADESCO FUNDO DE INVESTIMENTO RENDA FIXA IDKA PRÉ 2,Fundo de Renda Fixa,9.060080e+08
26.344.495/0001-60,5.088517,ITAÚ PRIVATE RENDA FIXA PRÉ IDKA 3 FICFI,Fundo de Renda Fixa,1.350196e+08
29.197.055/0001-15,4.945899,ITAÚ LEGEND RENDA FIXA LONGO PRAZO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo de Renda Fixa,3.521745e+09


In [0]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
01.671.480/0001-85,-26.927657,SANTOS VIRTUAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,4.406207e+06
05.357.507/0001-10,-14.023970,BANESTES INSTITUCIONAL FUNDO DE INVESTIMENTO RENDA FIXA,Fundo de Renda Fixa,1.964791e+08
31.095.989/0001-98,-9.976724,ITAÚ RENDA FIXA JUROS REAIS B5+ FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo de Renda Fixa,1.051130e+09
07.400.556/0001-14,-9.927680,ICATU VANGUARDA FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO INFLAÇÃO LONGA RF LP,Fundo de Renda Fixa,4.047801e+08
27.884.230/0001-17,-9.795351,ITAÚ PRIVATE RENDA FIXA IMA-B 5+ FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo de Renda Fixa,1.636591e+08
10.577.503/0001-88,-9.746506,FUNDO DE INVESTIMENTO CAIXA BRASIL IMA B 5+ TITULOS PÚBLICOS RENDA FIXA LONGO PRAZO,Fundo de Renda Fixa,1.935722e+09
20.374.752/0001-20,-9.687322,BTG PACTUAL TESOURO IPCA LONGO FUNDO DE INVESTIMENTO RENDA FIXA REFERENCIADO,Fundo de Renda Fixa,2.506169e+08
13.327.340/0001-73,-9.677096,BB PREVIDENCIÁRIO RENDA FIXA IMA-B5+ TÍTULOS PÚBLICOS FUNDO DE INVESTIMENTO,Fundo de Renda Fixa,1.957720e+09
10.264.241/0001-00,-9.673596,ITAÚ FLEXPREV VÉRTICE IMAB5 MAIS RENDA FIXA - FUNDO DE INVESTIMENTO,Fundo de Renda Fixa,3.541743e+09


###Fundos Cambiais: 10 Melhores de 10 Piores de 2020

In [0]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100, classe='cambial')

In [0]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
33.144.687/0001-98,43.460456,BTG PACTUAL DIGITAL DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.838957e+08
01.623.535/0001-81,43.153661,ITAÚ CAMBIAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,1.076893e+09
28.516.058/0001-01,43.140339,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL ÁGORA,Fundo Cambial,1.655603e+07
31.132.386/0001-19,43.114212,TREND DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.292850e+08
01.430.938/0001-04,43.096075,BTG PACTUAL DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.790911e+08
02.294.024/0001-26,43.085551,BRADESCO H FUNDO DE INVESTIMENTO CAMBIAL LONGO PRAZO DÓLAR,Fundo Cambial,2.627622e+08
04.244.149/0001-77,42.961802,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL DÓLAR SPECIAL,Fundo Cambial,9.366153e+07
23.731.760/0001-74,42.924450,ITAÚ PERSONNALITÉ CAMBIAL DÓLAR PLUS FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,1.196935e+08
05.114.733/0001-70,42.771349,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAIXA CAMBIAL DÓLAR,Fundo Cambial,1.310395e+08


In [0]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
32.246.789/0001-51,-29.891478,TREND SHORT DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,6.095320e+06
04.305.193/0001-40,38.568461,BB CAMBIAL EURO LP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,1.299703e+08
00.819.855/0001-49,39.422331,WESTERN ASSET CÂMBIO DOLAR SILVER FICFI CAMBIAL,Fundo Cambial,2.664146e+06
00.819.880/0001-22,40.816680,WESTERN ASSET CÂMBIO DOLAR PRIME FICFI CAMBIAL,Fundo Cambial,4.302020e+07
00.819.889/0001-33,40.856621,WESTERN ASSET HEDGE DÓLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.070803e+08
04.128.893/0001-06,41.930025,BB CAMBIAL DOLAR LP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,3.113414e+08
02.543.933/0001-50,41.940647,SANTANDER FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.056582e+08
04.061.044/0001-82,42.026233,BB CAMBIAL DÓLAR LP VIP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,6.060828e+08
01.171.956/0001-19,42.214357,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL DÓLAR,Fundo Cambial,1.424385e+07
